In [1]:
from pipython import GCSDevice, pitools, datarectools
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import * #QDoubleValidator
import sys
import os
import pyvisa
import pickle
import numpy as np
from time import sleep
import pandas as pd
import math

class Window(QWidget):
    
    def __init__(self):
        super().__init__()
        self.title = 'GUI'
        self.left = 0
        self.top = 0
        self.width = 640
        self.height = 480
        
        
        
        self.stepSize_x = 0
        self.stepSize_y = 0
        self.stepSize_z = 0
        
        self.Xposition = 0
        self.Yposition = 0
        self.Zposition = 0
        self.screen = None
        
        self.power_value1 = 0
        self.power_value2 = 0

        self.measurement_index = 0

        self.efficiency = 0.00
        
        self.initUI()
        
        
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)

        layout = QBoxLayout(QBoxLayout.LeftToRight)
        self.setLayout(layout)

        # Groupbox for Input Parameters
        groupbox = QGroupBox("Input Parameters")
        layout.addWidget(groupbox)
        vbox_input = QVBoxLayout()
        groupbox.setLayout(vbox_input)

        self.labelWavelength = QLabel('Wavelength (nm):', self)
        vbox_input.addWidget(self.labelWavelength)
        self.lineeditWavelength = QLineEdit()
        self.lineeditWavelength.setPlaceholderText('Enter WaveLength and Press Enter')
        self.lineeditWavelength.setValidator(QDoubleValidator())
        self.lineeditWavelength.returnPressed.connect(self.return_pressed_wavelentgth)
        vbox_input.addWidget(self.lineeditWavelength)

        self.labelaperture = QLabel('Aperture (mm):', self)
        vbox_input.addWidget(self.labelaperture)
        self.lineeditaperture = QLineEdit()
        self.lineeditaperture.setPlaceholderText('Enter aperture and Press Enter')
        self.lineeditaperture.setValidator(QDoubleValidator())
        self.lineeditaperture.returnPressed.connect(self.return_pressed_aperture)
        vbox_input.addWidget(self.lineeditaperture)
        
        # Groupbox for Actuator
        groupbox = QGroupBox("X-Axis")
        layout.addWidget(groupbox)
        vbox = QVBoxLayout()
        groupbox.setLayout(vbox)


        # Button for forward X-axis
        button_forwardXAxis = QPushButton("Forward X-Axis")
        button_forwardXAxis.clicked.connect(self.XAxisForward)
        button_forwardXAxis.setToolTip("This button is used for moving X-axis in forward direction ")
        
        vbox.addWidget(button_forwardXAxis)

        # Button for backward X-axis
        button_backwardXAxis = QPushButton("Backward X-Axis")
        button_backwardXAxis.clicked.connect(self.XAxisBackward)
        button_backwardXAxis.setToolTip("This button is used for moving X-axis in backward direction ")
        vbox.addWidget(button_backwardXAxis)

        # Step size block
        self.lineedit = QLineEdit()
        self.lineedit.setPlaceholderText('Enter Step Size and Press Enter')
        self.lineedit.setValidator(QDoubleValidator())
        self.lineedit.returnPressed.connect(self.return_pressed_x)
        vbox.addWidget(self.lineedit)

        self.labelX = QLabel('X-Axis Value: 0', self)
        vbox.addWidget(self.labelX)


        # Group box for yth axis
        groupbox = QGroupBox("Y-axis")
        layout.addWidget(groupbox)
        vbox2 = QVBoxLayout()
        groupbox.setLayout(vbox2)

        

         # Button for Forward Y-axis
        button_forwardYAxis = QPushButton("Forward Y-Axis")
        button_forwardYAxis.clicked.connect(self.YAxisForward)
        button_forwardYAxis.setToolTip("This button is used for moving Y-axis in Backward direction ")
        vbox2.addWidget(button_forwardYAxis)

        # Button for Backward Y-axis
        button_backwardYAxis = QPushButton("Backward Y-Axis")
        button_backwardYAxis.clicked.connect(self.YAxisBackward)
        button_backwardYAxis.setToolTip("This button is used for moving Y-axis in Forward direction ")
        vbox2.addWidget(button_backwardYAxis)
        

        # Step size block For Y -axis
        self.lineedit1 = QLineEdit()
        self.lineedit1.setPlaceholderText('Enter Step Size and Press Enter')
        self.lineedit1.setValidator(QDoubleValidator())
        self.lineedit1.returnPressed.connect(self.return_pressed_y)
        vbox2.addWidget(self.lineedit1)

        self.labelY = QLabel('Y-Axis Value: 0', self)
        vbox2.addWidget(self.labelY)

        # Group box for zth axis
        groupbox = QGroupBox("Z-axis")
        layout.addWidget(groupbox)
        vbox3 = QVBoxLayout()
        groupbox.setLayout(vbox3)


         # Button for forward Z-axis
        button_forwardZAxis = QPushButton("Forward Z-Axis")
        button_forwardZAxis.clicked.connect(self.ZAxisForward)
        button_forwardZAxis.setToolTip("This button is used for moving Z-axis in forward direction ")
        vbox3.addWidget(button_forwardZAxis)
        
        # Button for backward Z-axis
        button_backwardZAxis = QPushButton("Backward Z-Axis")
        button_backwardZAxis.clicked.connect(self.ZAxisBackward)
        button_backwardZAxis.setToolTip("This button is used for moving Z-axis in backward direction ")
        vbox3.addWidget(button_backwardZAxis)

        # Step size block For Z -axis
        self.lineedit2 = QLineEdit()
        self.lineedit2.setPlaceholderText('Enter Step Size and Press Enter')
        self.lineedit2.setValidator(QDoubleValidator())
        self.lineedit2.returnPressed.connect(self.return_pressed_z)
        vbox3.addWidget(self.lineedit2)

        self.labelZ = QLabel('Z-Axis Value: 0', self)
        vbox3.addWidget(self.labelZ)

         # Group box for Sensor Measurement
        groupbox = QGroupBox("Sensor Measurement")
        layout.addWidget(groupbox)
        vbox4 = QVBoxLayout()
        groupbox.setLayout(vbox4)


        # Button for Sensor Measurement
        button_sensor1 = QPushButton("Sensor_Measurement")
        button_sensor1.clicked.connect(self.power_measurement)
        button_sensor1.setToolTip("This button is used for Taking Sensor measurements")
        vbox4.addWidget(button_sensor1)
        
        self.value_list = []
        button_measure_button = QPushButton("Coordinate")
        button_measure_button.clicked.connect(self.on_measure_button_clicked)
        button_measure_button.setToolTip("This button is used for taking coordinates")
        vbox4.addWidget(button_measure_button)

        
        button_reference_measure = QPushButton("Reference_Measurement")
        button_reference_measure.clicked.connect(self.reference_measurement)
        button_reference_measure.setToolTip("This button is used for Reference measurements")
        vbox4.addWidget(button_reference_measure)

        button_Scan_AOI = QPushButton("DOE_Measurement")
        button_Scan_AOI.clicked.connect(self.scan_area)
        button_Scan_AOI.setToolTip("This button is used for DOE measurement")
        vbox4.addWidget(button_Scan_AOI)

        button_Efficiency_Measurement = QPushButton("Efficiency_Measurement")
        button_Efficiency_Measurement.clicked.connect(self.efficiency_measurement)
        button_Efficiency_Measurement.setToolTip("This button is used for Getting the list of AOI")
        vbox4.addWidget(button_Efficiency_Measurement)

        
        # self.labelreference = QLabel('Reference Ratio: 0', self)
        self.label1 = QLabel('Sensor 1 Value: 0', self)
        self.label2 = QLabel('Sensor 2 Value: 0', self)
        # vbox4.addWidget(self.labelreference)
        vbox4.addWidget(self.label1)
        vbox4.addWidget(self.label2)

        self.labelefficiency = QLabel('Effeciency: 0', self)
        vbox4.addWidget(self.labelefficiency)

    
        # self.labelWavelength = QLabel('Wavelength (nm):', self)
        # vbox4.addWidget(self.labelWavelength)
        # self.lineeditWavelength = QLineEdit()
        # self.lineeditWavelength.setPlaceholderText('Enter WaveLength and Press Enter')
        # self.lineeditWavelength.setValidator(QDoubleValidator())
        # self.lineeditWavelength.returnPressed.connect(self.return_pressed_wavelentgth)
        # vbox4.addWidget(self.lineeditWavelength)

        # self.labelaperture = QLabel('Aperture:', self)
        # vbox4.addWidget(self.labelaperture)
        # self.lineeditaperture = QLineEdit()
        # self.lineeditaperture.setPlaceholderText('Enter aperture and Press Enter')
        # self.lineeditaperture.setValidator(QDoubleValidator())
        # self.lineeditaperture.returnPressed.connect(self.return_pressed_aperture)
        # vbox4.addWidget(self.lineeditaperture)

    def show_popup_message(self, message):
         
        app = QApplication(sys.argv)
        
        # Create a QMessageBox
        message_box = QMessageBox()
        message_box.setWindowTitle("Task Completed")
        message_box.setText(message)
        message_box.setIcon(QMessageBox.Information)
        
        # Add a button (optional)
        message_box.addButton(QMessageBox.Ok)
        
        # Show the message box
        result = message_box.exec_()    

     # function for Stepsize Returning
    def return_pressed_x(self):
        self.stepSize_x = float(self.lineedit.text())
        return self.stepSize_x

    
    def XAxisForward(self):
        l = []
        ss = self.stepSize_x
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
            
            
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.

            pidevice.ConnectUSB(serialnum='0115500179')
            pidevice.SVO('1',1)
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):

                l.append(positions[axis])
                print(l)
            incrementedList = [z + ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]
            print('input for forward',rangemax)

            pidevice.MOV('1', rangemax)

            # pitools.waitontarget(pidevice)
            sleep(0.5)
            positions = pidevice.qPOS('1')
            #print('Positions',positions['1'])
            print('updated position of {} = {:.2f}'.format('X-axis', positions['1']))
            # position = []
            # for axis in pidevice.qPOS('1'):
            #     print('updated position of axis {} = {:.2f}'.format('X-axis', positions[axis]))
            #     position.append(positions[axis])
            #     print('Test',position)

            
            self.Xposition = round(float(positions['1']),2)
            self.labelX.setText(f'X-Axis Value: {self.Xposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)

    
    def XAxisBackward(self):

        l = []
        ss = self.stepSize_x
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.

            pidevice.ConnectUSB(serialnum='0115500179')
            # positions = pidevice.qPOS('1')
            # for axis in pidevice.qPOS('1'):

            #     l.append(positions[axis])
            #     print(l)
            # incrementedList = [z - ss for z in l]
            # print(incrementedList)
            # rangemax = [ '%.5f' % elem for elem in incrementedList ]
            positions = pidevice.qPOS('1')
            
            rangemax = positions['1'] - ss
            print('input for movement',rangemax)
            pidevice.MOV('1', rangemax)

            # pitools.waitontarget(pidevice)
            sleep(0.5)
            positions = pidevice.qPOS('1')
            print('updated position of {} = {:.2f}'.format('X-axis', positions['1']))
            # position = []
            # for axis in pidevice.qPOS('1'):
            #     print('updated position of axis {} = {:.2f}'.format('X-axis', positions[axis]))
            #     position.append(positions[axis])
            #     print('Test',position)

            # strPositons = str(positions['1'])
            self.Xposition = round(float(positions['1']),2)
            self.labelX.setText(f'X-Axis Value: {self.Xposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)


     # function for Stepsize Returning
    def return_pressed_y(self):
        self.stepSize_y = float(self.lineedit1.text())
        return self.stepSize_y
    def YAxisForward(self):
        
        l = []
        ss = self.stepSize_y
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.DD']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.
            
            pidevice.ConnectUSB(serialnum='0135500452')
            
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):

                l.append(positions[axis])
                print(l)
            incrementedList = [z + ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            # pitools.waitontarget(pidevice)
            sleep(0.5)

            positions = pidevice.qPOS('1')
            print('updated position of axis {} = {:.2f}'.format('Y-axis', positions['1']))
            # for axis in pidevice.qPOS('1'):
            #     print('updated position of axis {} = {:.2f}'.format('Y-axis', positions[axis]))
            #     position.append(positions[axis])

            # strPositons = str(positions['1'])
            self.Yposition = round(float(positions['1']),2)
            self.labelY.setText(f'Y-Axis Value: {self.Yposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)
        
    def YAxisBackward(self):

        l = []
        ss = self.stepSize_y
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.DD']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.
            
            pidevice.ConnectUSB(serialnum='0135500452')
            
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):

                l.append(positions[axis])
                print(l)
            incrementedList = [z - ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            # pitools.waitontarget(pidevice)
            sleep(0.5)

            positions = pidevice.qPOS('1')
            print('updated position of axis {} = {:.2f}'.format('Y-axis', positions['1']))
            
            # position = []
            # for axis in pidevice.qPOS('1'):
            #     print('updated position of axis {} = {:.2f}'.format('Y-axis', positions[axis]))
            #     position.append(positions[axis])

            # strPositons = str(positions['1'])
            self.Yposition = round(float(positions['1']),2)
            self.labelY.setText(f'Y-Axis Value: {self.Yposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)


     # function for Stepsize Returning
    def return_pressed_z(self):
        self.stepSize_z = float(self.lineedit2.text())
        return self.stepSize_z
    
    def ZAxisForward(self):
        l = []
        ss = self.stepSize_z
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.

            pidevice.ConnectUSB(serialnum='0095500784')
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):
                
                l.append(positions[axis])
                print(l)
            incrementedList = [z + ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            # pitools.waitontarget(pidevice)
            sleep(0.5)

            positions = pidevice.qPOS('1')
            print('updated position of axis {} = {:.2f}'.format('Z-axis', positions['1']))
            # for axis in pidevice.qPOS('1'):
            #     print('updated position of axis {} = {:.2f}'.format('Z-axis', positions[axis]))
            #     position.append(positions[axis])
                
            # strPositons = str(positions['1'])
            self.Zposition = round(float(positions['1']),2)
            self.labelZ.setText(f'Z-Axis Value: {self.Zposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)
    def ZAxisBackward(self):

        l = []
        ss = self.stepSize_z
        print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.

            pidevice.ConnectUSB(serialnum='0095500784')
            positions = pidevice.qPOS('1')
            for axis in pidevice.qPOS('1'):

                l.append(positions[axis])
                print(l)
            incrementedList = [z - ss for z in l]
            print(incrementedList)
            rangemax = [ '%.5f' % elem for elem in incrementedList ]


            pidevice.MOV('1', rangemax)

            # pitools.waitontarget(pidevice)
            sleep(0.5)

            positions = pidevice.qPOS('1')
            print('updated position of axis {} = {:.2f}'.format('Z-axis', positions['1']))
            # for axis in pidevice.qPOS('1'):
            #     print('updated position of axis {} = {:.2f}'.format('Z-axis', positions[axis]))
            #     position.append(positions[axis])

            # strPositons = str(positions['1'])
            self.Zposition = round(float(positions['1']),2)
            self.labelZ.setText(f'Z-Axis Value: {self.Zposition}')
            #QMessageBox.information(self,'Actuator axes information', "axes moved forward to positions : "+ strPositons)


    def power_measurement(self):
        rm = pyvisa.ResourceManager()
        # List all connected resources
        # print("Resources detected\n{}\n".format(rm.list_resources()))
        # print("Opening connection...")
        instrument = rm.open_resource('USB0::0x1313::0x8022::M00533936::INSTR', timeout=5000)
        
        # print("Sending command...")
        command1 = ':POW1:VAL?'
        self.power_value1 = instrument.query(command1)
        command2 = ':POW2:VAL?'
        self.power_value2 = instrument.query(command2)
        
        # print(f"Sensor 1: {self.power_value1}")
        # print(f"Sensor 2: {self.power_value2}")
        
        # print("Closing connection...")
        instrument.close()
        self.label1.setText(f'Sensor_1 Value: {self.power_value1}')
        self.label2.setText(f'Sensor_2 Value: {self.power_value2}')
        return self.power_value1, self.power_value2
        
    def reference_measurement(self):
        self.reference = float(self.power_value1)/float(self.power_value2)
        self.labelreference.setText(f'Refernece Value: {self.reference}')
        return self.reference


    # function for Stepsize Returning
    def return_pressed_wavelentgth(self):
        self.wavelength = float(self.lineeditWavelength.text())
        self.wavelength = self.wavelength * (1E-9)
        return self.wavelength

    def on_measure_button_clicked(self):
        
        measurement = {
            'measurement_index': self.measurement_index,
            'X-axis': self.Xposition,
            'Y-axis': self.Yposition,
            'Z-axis': self.Zposition,
            'Ch1': self.power_value1,
            'Ch2': self.power_value2,
            'Aperture': self.aperture
        }
        self.value_list.append(measurement)
        # print("Measurement appended to the list:", measurement)

        print("list:", self.value_list)

        # Increment measurement index for the next measurement
        self.measurement_index += 1

        with open('Coordinates.pkl', 'wb') as file:
            pickle.dump(self.value_list, file)

    # function for Stepsize Returning
    def return_pressed_aperture(self):
        self.aperture = float(self.lineeditaperture.text())
        return self.aperture

    def XAxisForward_scan(self,step):
        
        # print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
            
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.
        
            pidevice.ConnectUSB(serialnum='0115500179')
            current_position = pidevice.qPOS('1')
            
            rangemax = current_position['1'] + step
    
    
            pidevice.MOV('1', rangemax)
            # pitools.waitontarget(pidevice)
            sleep(0.5)
            updated_position = pidevice.qPOS('1')
            
            return current_position['1'],updated_position['1']
    

            

    def XAxisBackward_scan(self,step):
        
        
    
        # print('step Size', ss)
        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.PD1']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
            
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.
    
            pidevice.ConnectUSB(serialnum='0115500179')
            current_position = pidevice.qPOS('1')
            rangemax = current_position['1'] - step
    
    
            pidevice.MOV('1', rangemax)
            # pitools.waitontarget(pidevice)
            sleep(0.5)
            updated_position = pidevice.qPOS('1')
            return current_position['1'],updated_position['1']

    def YAxisForward_scan(self, step):
        

        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.DD']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.
            
            pidevice.ConnectUSB(serialnum='0135500452')
            current_position = pidevice.qPOS('1')
            rangemax = current_position['1'] + step
    
    
            pidevice.MOV('1', rangemax)
            # pitools.waitontarget(pidevice)
            sleep(0.5)
            updated_position = pidevice.qPOS('1')
            return current_position['1'],updated_position['1']
            
    def YAxisBackward_scan(self,step):

        CONTROLLERNAME = 'C-863'  # 'C-884' will also work
        STAGES = ['M-521.DD']
        REFMODE = ('REF')
        with GCSDevice(CONTROLLERNAME) as pidevice:
        # InterfaceSetupDlg() is an interactive dialog. There are other methods to
        # connect to an interface without user interaction.
            
            pidevice.ConnectUSB(serialnum='0135500452')
            current_position = pidevice.qPOS('1')
            rangemax = current_position['1'] - step
    
    
            pidevice.MOV('1', rangemax)
            # pitools.waitontarget(pidevice)
            sleep(0.5)
            updated_position = pidevice.qPOS('1')
            return current_position['1'],updated_position['1']

    def reference_measurement(self):

        def frange(start, stop, step):
            while start < stop:
                yield start
                start += step
        
        measurement_list = []
        
        # list1 = loaded_list # for testing
        with open('Coordinates.pkl', 'rb') as f:
            list1 = pickle.load(f) # for actual
            
        x1 = list1[0]['X-axis']
        # x1 = float(x1.strip('[]'))
        print('Starting X Axis position',x1)
        x2 = list1[1]['X-axis']
        # x2 = float(x2.strip('[]'))
        print('Ending X-Axis position',x2)
        diff_x = round((x2-x1),2)
        # print('diff_x',diff_x)
        y1 = list1[0]['Y-axis']
        # y1 = float(y1.strip('[]'))
        y4 = list1[2]['Y-axis']
        # y4 = float(y4.strip('[]'))
        print('Starting Y Axis position',y1)
        print('Ending Y-Axis position',y4)
        diff_y = round((y4-y1),2)
        # print('diff_y',diff_y)
        x_step = list1[0]['Aperture']
        # print('x_step',x_step)
        y_step = list1[0]['Aperture']
        print('Loop will b starting starting')
        current_y_position = y1
        updated_y_position = y1
        for y in np.arange(0, (diff_y+y_step),y_step):
        
            if y<=diff_y:
              
                for x in np.arange(0, (diff_x+x_step),x_step):
                    
                    if x<=diff_x:
                        
                        # print('Current X-axis Position',x)
                        current_x_position, updated_x_position = self.XAxisForward_scan(x_step)
                        a,b = self.power_measurement()
                        measurement = {
                        'Current X-axis': round(current_x_position,2),
                            # 'Updated X-axis': updated_x_position,    
                        'Current Y-axis': round(updated_y_position,2),
                            # 'Updated Y-axis': updated_y_position,
                        'Ch1': a,
                        'Ch2': b
                    }
                        
                        measurement_list.append(measurement)
                        print(measurement)
                    else:
                        break
                    # print(effeciency_list)
                    # print('Updated X-axis Position',x+x_step)
                  
                self.XAxisBackward_scan(diff_x+x_step)
                print('Xaxis moved backward')
                
                current_y_position, updated_y_position = self.YAxisForward_scan(y_step)
                print('Yaxis moved forward one step')
             
        self.YAxisBackward_scan(diff_y+y_step)
        print('Yaxis moved to original position')
        
        with open('Measurement_Reference.pkl', 'wb') as file:
            pickle.dump(measurement_list, file)
        QMessageBox.information(self,'Measurement Information','Reference Measurement Done')
    def scan_area(self):
          

        def frange(start, stop, step):
            while start < stop:
                yield start
                start += step
        
        measurement_list = []
        
        # list1 = loaded_list # for testing
        with open('Coordinates.pkl', 'rb') as f:
            list1 = pickle.load(f) # for actual
            
        x1 = list1[0]['X-axis']
        # x1 = float(x1.strip('[]'))
        print('Starting X Axis position',x1)
        x2 = list1[1]['X-axis']
        # x2 = float(x2.strip('[]'))
        print('Ending X-Axis position',x2)
        diff_x = round((x2-x1),2)
        # print('diff_x',diff_x)
        y1 = list1[0]['Y-axis']
        # y1 = float(y1.strip('[]'))
        y4 = list1[2]['Y-axis']
        # y4 = float(y4.strip('[]'))
        print('Starting Y Axis position',y1)
        print('Ending Y-Axis position',y4)
        diff_y = round((y4-y1),2)
        # print('diff_y',diff_y)
        x_step = list1[0]['Aperture']
        # print('x_step',x_step)
        y_step = list1[0]['Aperture']
        print('Loop will b starting starting')
        current_y_position = y1
        updated_y_position = y1
        for y in np.arange(0, (diff_y+y_step),y_step):
        
            if y<=diff_y:
              
                for x in np.arange(0, (diff_x+x_step),x_step):
                    
                    if x<=diff_x:
                        
                        # print('Current X-axis Position',x)
                        current_x_position, updated_x_position = self.XAxisForward_scan(x_step)
                        a,b = self.power_measurement()
                        measurement = {
                        'Current X-axis': round(current_x_position,2),
                            # 'Updated X-axis': updated_x_position,    
                        'Current Y-axis': round(updated_y_position,2),
                            # 'Updated Y-axis': updated_y_position,
                        'Ch1': a,
                        'Ch2': b
                    }
                        
                        measurement_list.append(measurement)
                        print(measurement)
                    else:
                        break
                    # print(effeciency_list)
                    # print('Updated X-axis Position',x+x_step)
                  
                self.XAxisBackward_scan(diff_x+x_step)
                print('Xaxis moved backward')
                
                current_y_position, updated_y_position = self.YAxisForward_scan(y_step)
                print('Yaxis moved forward one step')
             
        self.YAxisBackward_scan(diff_y+y_step)
        print('Yaxis moved to original position')
        with open('Measurement_DOE.pkl', 'wb') as file:
            pickle.dump(measurement_list, file)
        QMessageBox.information(self,'Measurement Information','DOE Measurement Done')

    def efficiency_measurement(self):
        with open('Measurement_DOE.pkl', 'rb') as f:
            Measurement_DOE = pickle.load(f)
        with open('Measurement_Reference.pkl', 'rb') as f:
            Measurement_Reference = pickle.load(f)
        key_sensor= 'Ch1'
        sensor_value_Reference = [float(d[key_sensor]) for d in Measurement_Reference]
        sum_sensor_Reference = sum(sensor_value_Reference)
        sensor_value_DOE = [float(d[key_sensor]) for d in Measurement_DOE]
        sum_sensor_DOE = sum(sensor_value_DOE)
        self.efficiency = sum_sensor_Reference/sum_sensor_DOE
        print(f'Efficiency:',round(self.efficiency,3))
        self.labelefficiency.setText(f'Efficiency Value: {self.efficiency}')
        

app = QApplication(sys.argv)
screen = Window()
screen.show()
try:
    sys.exit(app.exec_())
except SystemExit:    
    print('Good Luck!')      

step Size 0
[105.0]
[105.0]
input for forward ['105.00000']
updated position of X-axis = 105.00
step Size 0
[102.0]
[102.0]
updated position of axis Y-axis = 102.00
step Size 0
[102.0]
[102.0]
updated position of axis Z-axis = 102.00
list: [{'measurement_index': 0, 'X-axis': 105.0, 'Y-axis': 102.0, 'Z-axis': 102.0, 'Ch1': '1.02022636E-06\n', 'Ch2': '3.89418801E-06\n', 'Aperture': 0.5}]
step Size 2.0
[105.0]
[107.0]
input for forward ['107.00000']
updated position of X-axis = 107.00
list: [{'measurement_index': 0, 'X-axis': 105.0, 'Y-axis': 102.0, 'Z-axis': 102.0, 'Ch1': '1.02022636E-06\n', 'Ch2': '3.89418801E-06\n', 'Aperture': 0.5}, {'measurement_index': 1, 'X-axis': 107.0, 'Y-axis': 102.0, 'Z-axis': 102.0, 'Ch1': '1.02022636E-06\n', 'Ch2': '3.89418801E-06\n', 'Aperture': 0.5}]
step Size 2.0
[102.0]
[104.0]
updated position of axis Y-axis = 104.00
list: [{'measurement_index': 0, 'X-axis': 105.0, 'Y-axis': 102.0, 'Z-axis': 102.0, 'Ch1': '1.02022636E-06\n', 'Ch2': '3.89418801E-06\n', 'A